# Adversarial logits pairing (ALP)

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '0,1,2,3'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/alp/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.01
momentum = 0.9
weight_decay = 0.0035
batch_size = 128
total_epochs = 100
epsilon = 8/255
alpha = 2/255
num_repeats = 10
lam = 0.5

## Inner maximization

In [3]:
def inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats):
    noise = torch.FloatTensor(inputs.shape).uniform_(-1, 1).cuda()
    x = torch.clamp(inputs + noise, min=0, max=1)
    for _ in range(num_repeats):
        x.requires_grad_()
        logits = model(x)
        loss = xent(logits, targets)
        loss.backward()
        grads = x.grad.data
        x = x.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs - epsilon), inputs + epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def training(epoch, model, dataloader, optimizer, num_classes, 
             epsilon=8/255, alpha=2/255, num_repeats=10, lam=0.5):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
        logits_adv = model(x)
        logits_nat = model(inputs)
        
        loss_xent = xent(logits_adv, targets)
        loss_lp = torch.norm(logits_nat - logits_adv, p=2, dim=-1).sum()/batch
        loss = loss_xent + lam*loss_lp
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/(idx+1),
                     num_correct/batch, total_correct/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
            
            x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
    
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(),lr=lr, momentum=momentum)

scheduler = [75, 90]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes, epsilon, alpha, num_repeats, lam)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)
        
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 3.3305 (avg: 3.3305) | acc: 0.0000 (avg: 0.0000) |
Epoch 0 [100/383] | loss: 2.3102 (avg: 2.4285) | acc: 0.1562 (avg: 0.1143) |
Epoch 0 [200/383] | loss: 2.2877 (avg: 2.3627) | acc: 0.1641 (avg: 0.1391) |
Epoch 0 [300/383] | loss: 2.1955 (avg: 2.3338) | acc: 0.2031 (avg: 0.1524) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:37<00:00,  4.60it/s, acc_nat=0.231, acc_rob=0.197]


Epoch 1 [0/383] | loss: 2.2477 (avg: 2.2477) | acc: 0.1875 (avg: 0.1875) |
Epoch 1 [100/383] | loss: 2.2308 (avg: 2.2499) | acc: 0.2344 (avg: 0.1943) |
Epoch 1 [200/383] | loss: 2.1912 (avg: 2.2426) | acc: 0.1797 (avg: 0.1966) |
Epoch 1 [300/383] | loss: 2.2142 (avg: 2.2368) | acc: 0.2344 (avg: 0.2006) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:34<00:00,  4.65it/s, acc_nat=0.235, acc_rob=0.207]


Epoch 2 [0/383] | loss: 2.1902 (avg: 2.1902) | acc: 0.2109 (avg: 0.2109) |
Epoch 2 [100/383] | loss: 2.2109 (avg: 2.2183) | acc: 0.2734 (avg: 0.2098) |
Epoch 2 [200/383] | loss: 2.2176 (avg: 2.2148) | acc: 0.2031 (avg: 0.2160) |
Epoch 2 [300/383] | loss: 2.2350 (avg: 2.2091) | acc: 0.2188 (avg: 0.2201) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [03:20<00:00,  4.99it/s, acc_nat=0.723, acc_rob=0.49]


Epoch 98 [0/383] | loss: 1.4203 (avg: 1.4203) | acc: 0.6250 (avg: 0.6250) |
Epoch 98 [100/383] | loss: 1.5809 (avg: 1.5010) | acc: 0.6016 (avg: 0.5832) |
Epoch 98 [200/383] | loss: 1.4985 (avg: 1.4971) | acc: 0.6094 (avg: 0.5859) |
Epoch 98 [300/383] | loss: 1.6251 (avg: 1.4965) | acc: 0.5078 (avg: 0.5849) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:22<00:00,  4.94it/s, acc_nat=0.734, acc_rob=0.5] 


Epoch 99 [0/383] | loss: 1.5775 (avg: 1.5775) | acc: 0.5391 (avg: 0.5391) |
Epoch 99 [100/383] | loss: 1.5427 (avg: 1.5030) | acc: 0.6172 (avg: 0.5822) |
Epoch 99 [200/383] | loss: 1.4621 (avg: 1.4990) | acc: 0.6328 (avg: 0.5850) |
Epoch 99 [300/383] | loss: 1.4357 (avg: 1.4965) | acc: 0.6641 (avg: 0.5843) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:20<00:00,  4.99it/s, acc_nat=0.72, acc_rob=0.506]
